#Downloading Solar Data with SunPy

This lesson covers the functionality in SunPy to download solar physics data from various places. It has been written for the current release (v0.5.4), despite the fact there are large improvments in the downloading software for SunPy v0.6.

This lesson will cover the core downloaders availible in the `sunpy.net` submodule, we will cover the following topics:

* The Virtual Solar Observatory.
* The SunPy query syntax.
* Downloading data and managing files.
* JSOC Queries.
* The SunPy database.

## The Virtual Solar Observatory

The VSO is a centerlised portal to many different sources of solar physics data. SunPy has a very powerful and easy to use VSO client, which we are going to use to download an AIA image.

Import the `vso` module:

In [ ]:
from sunpy.net import vso

Create a client instance:

In [ ]:
vclient = vso.VSOClient()

Query the VSO:

In [ ]:
results = vclient.query(vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01') & vso.attrs.Instrument('AIA'))
results.show()

Download the File:

In [ ]:
files = vclient.get(results).wait(progress=True)
files

## SunPy Query Syntax

Having done a quick demo of the VSO client, we will no go through the steps in more detail and explain them. The  main part of the VSO interface is the query syntax.
The query syntax allows you to perform very complex queries, by using the logical and `&` and logical or `|` operators on the indivual query attributes.

The query attributes live in the `sunpy.net.vso.attrs` namespace, which we have imported as `vso.attrs`.

The attributes availible for use in a VSO query are:

In [ ]:
vso.attrs.__all__

You can combine these attributes to make simple or more complex queries.
First, let's explain the example we used earlier:

`vclient.query(vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01') & vso.attrs.Instrument('AIA'))`

In [ ]:
vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:00:01')

The `Time` attribute, specifies a time range for the data, you must specify a start and an end value to `Time`.

In [ ]:
vso.attrs.Instrument('AIA')

The `Instrument` attribute specifies the name of the instrument used to collect the data, the list of availible instruments is on the VSO website:

When we passed these into `query()` we used the `&` symbol to tell `query()` that we wanted VSO to look for records that satisfied both attributes.

It is also possible to use the `|` operator to get multiple results:

In [ ]:
vclient.query(vso.attrs.Time('2015/03/15T00:00:29', '2015/03/15T00:00:30')
              & (vso.attrs.Instrument('AIA') | vso.attrs.Instrument('HMI'))).show()

We can also construct these queries outside of the `query()` method, to make it easier to read or to make multiple queries:

In [ ]:
time = vso.attrs.Time('2015/03/15T00:00:00', '2015/03/15T00:01:00')
aia = vso.attrs.Instrument('AIA') & vso.attrs.Sample('60')
hmi = vso.attrs.Instrument('HMI') & vso.attrs.Physobs('LOS_magnetic_field')

In [ ]:
vclient.query(time, aia | hmi).show()

## Retreiving Files

So far we have been downloading files using `get()` with the default options. In this section we are going to look at how to customise these options and save the files to different directories.

In [ ]:
results = vclient.query(time, aia | hmi)
results.show()

Once we have obtained the results we can download them with the `get()` method:

In [ ]:
help(vclient.get)